In [1]:
!pip install flask flask-ngrok tensorflow pandas numpy scikit-learn joblib

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load data
train_df = pd.read_csv("train.csv")
train_df['date'] = pd.to_datetime(train_df['date'], errors='coerce')

# Normalize sales
scaler = MinMaxScaler()
train_df['sales_scaled'] = scaler.fit_transform(train_df[['sales']])

# Save scaler for later use
import joblib
joblib.dump(scaler, 'scaler.pkl')

# Display first 5 rows
train_df.head()

,date,store,item,sales,sales_scaled
0,2013-01-01,1.0,1.0,13.0,0.056277
1,2013-01-02,1.0,1.0,11.0,0.047619
2,2013-01-03,1.0,1.0,14.0,0.060606
3,2013-01-04,1.0,1.0,13.0,0.056277
4,2013-01-05,1.0,1.0,10.0,0.043290


In [4]:
SEQ_LENGTH = 30

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i: i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

X_train, y_train = [], []
for (store, item), group in train_df.groupby(['store', 'item']):
    sales_seq = group['sales_scaled'].values
    X_seq, y_seq = create_sequences(sales_seq, SEQ_LENGTH)
    X_train.append(X_seq)
    y_train.append(y_seq)

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_train = np.expand_dims(X_train, axis=-1)  # Reshape for LSTM

X_train.shape, y_train.shape

((438721, 30, 1), (438721,))

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

# Define the model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    BatchNormalization(),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    BatchNormalization(),
    Dense(25, activation="relu"),
    Dense(1, activation="linear")
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Save the trained model
model.save("inventory_lstm_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 55s 9ms/step - loss: 0.0307 - val_loss: 0.0034
Epoch 2/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 76s 8ms/step - loss: 0.0031 - val_loss: 0.0016
Epoch 3/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - loss: 0.0018 - val_loss: 0.0016
Epoch 4/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 83s 9ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 5/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 81s 8ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 6/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 7/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 81s 8ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 8/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 9/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 10/10
5484/5484 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - loss: 0.0014 - val_loss: 0.0014


In [6]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import joblib

from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import joblib
from tensorflow.keras.losses import MeanSquaredError # Import MeanSquaredError

app = Flask(__name__)

# Load trained model and scaler
model = load_model("inventory_lstm_model.h5", custom_objects={"mse": MeanSquaredError()}) # Use imported MeanSquaredError
model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
scaler = joblib.load("scaler.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    store = data['store']
    item = data['item']
    date = pd.to_datetime(data['date'])

    # Get last 30 days of sales for store-item
    past_data = train_df[(train_df['store'] == store) & (train_df['item'] == item)].tail(SEQ_LENGTH)

    if len(past_data) < SEQ_LENGTH:
        return jsonify({"error": "Not enough historical data"}), 400

    past_sales = past_data['sales_scaled'].values.reshape(1, SEQ_LENGTH, 1)

    # Predict sales
    predicted_sales_scaled = model.predict(past_sales)[0][0]
    predicted_sales = scaler.inverse_transform([[predicted_sales_scaled]])[0][0]

    return jsonify({"store": store, "item": item, "date": str(date.date()), "predicted_sales": round(predicted_sales, 2)})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [7]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

run_with_ngrok(app)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [9]:
import requests

url = "https://xyz.ngrok.io/predict"  # Make sure this is the correct ngrok URL
data = {
    "store": 1,
    "item": 1,
    "date": "2018-01-01"
}

response = requests.post(url, json=data)

if response.status_code == 200:
    print(response.json())
else:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

Request failed with status code 404
<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static